In [142]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.feature_extraction import FeatureHasher

In [158]:
defaultHiperparametrosClusteringAvisos = { 'cantidadFeaturesTitulo' : 100, 'cantidadClusteresAvisos' : 20, 'featureListParaClustering': ['tipo_de_trabajo_nro', 'nivel_laboral_nro', 'nombre_area_nro', 'zona_nro'], 'randomState': 177 }

def crear_hiperparametro_avisos(cantidadClusteresAvisos = 20, cantidadFeaturesTitulo = 100, featureListParaClustering = [], randomState = 177):
    hiperparametros = { 'cantidadFeaturesTitulo' : cantidadFeaturesTitulo, 'cantidadClusteresAvisos' : cantidadClusteresAvisos, 'featureListParaClustering': featureListParaClustering, 'randomState': randomState}
    return hiperparametros

def agregar_clusters(avisos_detalle, hiperparametrosAvisos = None):
    if(hiperparametrosAvisos is None):
        hiperparametrosAvisos = defaultHiperparametrosClusteringAvisos
    x_avisos = hiperparametrosAvisos['featureListParaClustering']
    if(hiperparametrosAvisos['cantidadFeaturesTitulo']>0):
        x_avisos= x_avisos + list(range(0, hiperparametrosAvisos['cantidadFeaturesTitulo']))
        h = FeatureHasher(n_features = hiperparametrosAvisos['cantidadFeaturesTitulo'], non_negative=True, input_type='string', dtype='int')
        x = h.transform(avisos_detalle['titulo'])
        avisos_detalle['titulo'] = list(x.toarray())
        titulos_como_lista = avisos_detalle.titulo.apply(pd.Series)
        avisos_detalle = pd.merge(avisos_detalle, titulos_como_lista, left_index = True, right_index = True)
        avisos_detalle = avisos_detalle.drop(['titulo'], axis=1)
    if(hiperparametrosAvisos['cantidadClusteresAvisos']>0):
        y_pred = KMeans(n_clusters=hiperparametrosAvisos['cantidadClusteresAvisos'], random_state=hiperparametrosAvisos['randomState']).fit_predict(avisos_detalle[x_avisos])
        avisos_detalle['cluster_aviso'] = y_pred
    return avisos_detalle

def columnas_para_leer_avisos():
    return ['descripcion', 'nombre_zona', 'tipo_de_trabajo', 'nivel_laboral', 'titulo_aviso','nombre_area','denominacion_empresa', 'cluster_aviso']

def get_avisos_detalle(hiperparametrosAvisos = None):    
    avisos1 = pd.read_csv('data/fiuba_6_avisos_detalle.csv')
    avisos2 = pd.read_csv('data/FiubaDesde15Abril/fiuba_6_avisos_detalle.csv')
    avisos3 = pd.read_csv('data/FiubaHasta15Abril/fiuba_6_avisos_detalle.csv')
    avisos4 = pd.read_csv('data/fiuba_6_avisos_detalle_missing_nivel_laboral.csv')        
    avisos_detalle = pd.concat([avisos1,avisos2,avisos3,avisos4]).drop_duplicates(subset=['idaviso'], keep='last').reset_index(drop=True)
    columns_rename = {'idpostulante': 'id_postulante', 'idaviso': 'id_aviso'}
    avisos_detalle = avisos_detalle.rename(columns=columns_rename)
    to_nivel_laboral_nro = {'Senior / Semi-Senior' : 2, 'Junior':1, 'Otro':0,
       'Jefe / Supervisor / Responsable':3,
       'Gerencia / Alta Gerencia / Dirección':4}
    to_tipo_trabajo_nro={'Full-time':0, 'Part-time':1, 'Teletrabajo':2, 'Por Horas':3, 'Pasantia':4,
       'Temporario':5, 'Por Contrato':6, 'Fines de Semana':7, 'Primer empleo':8,
       'Voluntario':9}
    to_nombre_area_numero = pd.Series(avisos_detalle['nombre_area'].unique()).to_dict()
    to_nombre_area_numero  = {v: k for k, v in to_nombre_area_numero.items()}
    to_nombre_zona_numero = pd.Series(avisos_detalle['nombre_zona'].unique()).to_dict()
    to_nombre_zona_numero  = {v: k for k, v in to_nombre_zona_numero.items()}
    avisos_detalle['nivel_laboral_nro']= avisos_detalle['nivel_laboral'].map(to_nivel_laboral_nro)
    avisos_detalle['nivel_laboral_nro'].fillna(value =0, inplace = True)
    avisos_detalle['tipo_de_trabajo_nro'] = avisos_detalle['tipo_de_trabajo'].map(to_tipo_trabajo_nro)
    avisos_detalle['nombre_area_nro'] = avisos_detalle['nombre_area'].map(to_nombre_area_numero)
    avisos_detalle['zona_nro'] = avisos_detalle['nombre_zona'].map(to_nombre_zona_numero)
    avisos_detalle['titulo_aviso'] = avisos_detalle['titulo']    
    
    return agregar_clusters(avisos_detalle, hiperparametrosAvisos)

In [159]:
avisos = get_avisos_detalle(crear_hiperparametro_avisos(cantidadClusteresAvisos=90, featureListParaClustering=['nombre_area_nro']))

C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\sklearn\feature_extraction\hashing.py:94: DeprecationWarning: the option non_negative=True has been deprecated in 0.19 and will be removed in version 0.21.
  " in version 0.21.", DeprecationWarning)


In [160]:
avisos['cluster_aviso'].value_counts()

8     700
72    682
23    658
0     649
17    596
51    589
25    581
64    579
53    578
83    555
9     525
6     520
59    506
21    470
50    456
75    444
29    439
66    405
28    395
27    393
55    383
70    383
80    372
52    363
35    362
31    355
24    351
37    351
61    343
42    333
     ... 
30    175
56    171
32    169
3     162
45    161
57    160
40    155
68    153
65    152
14    150
73    150
15    145
11    136
2     119
16    118
20    116
60    115
33    112
4     109
44    104
82     98
78     88
19     82
43     78
10     72
81     56
48     54
34     53
69     46
77     43
Name: cluster_aviso, Length: 90, dtype: int64

In [162]:
cluster_a_chequear = 77
avisos[avisos['cluster_aviso'] == cluster_a_chequear][columnas_para_leer_avisos()]

,descripcion,nombre_zona,tipo_de_trabajo,nivel_laboral,titulo_aviso,nombre_area,denominacion_empresa,cluster_aviso
2878,"<p style=""""><span style=""""><span style=""""><spa...",Gran Buenos Aires,Full-time,Senior / Semi-Senior,Farmacéutico auxiliar con Bloqueo para Importa...,Farmacia comercial,IEC,77
2985,"<p><span style="""">Para inscribirse en el torne...",Gran Buenos Aires,Teletrabajo,Gerencia / Alta Gerencia / Dirección,21/04: Online Hiring Tournament - VP of Profes...,Media Planning,CrossOver,77
3187,<p>Nos encontramos en la búsqueda de un <stron...,Gran Buenos Aires,Full-time,Senior / Semi-Senior,Coordinador Medico para Importante Sanatorio C...,Emergentología,GMS S.A.,77
3561,<p>Compañía multinacional de bombas centrífuga...,Gran Buenos Aires,Full-time,Senior / Semi-Senior,ING. DE VENTAS DE REPUESTOS Y SERVICIOS DE REP...,Ingeniería de Ventas,LG Recursos Humanos.,77
3562,<p><strong>Compañía multinacional en franco cr...,Gran Buenos Aires,Full-time,Senior / Semi-Senior,INGENIERO DE VENTAS - EQUIPAMIENTO DE GRAN POR...,Ingeniería de Ventas,LG Recursos Humanos.,77
3563,<p><strong>Compañía multinacional en franco cr...,Gran Buenos Aires,Full-time,Senior / Semi-Senior,INGENIERO DE VENTAS - EQUIPAMIENTO DE GRAN POR...,Ingeniería de Ventas,LG Recursos Humanos.,77
3564,<p>Compañía multinacional de bombas centrífuga...,Gran Buenos Aires,Full-time,Senior / Semi-Senior,INGENIERO DE VENTAS Sr. - Unidad de Sistemas d...,Ingeniería de Ventas,LG Recursos Humanos.,77
4757,"<p style=""""><span style="""">Empresa de comunica...",Gran Buenos Aires,Full-time,Senior / Semi-Senior,Traductor Público/Técnico-Científico-Literario...,Traduccion,Agencia Medica,77
5175,"<p><span style="""">Importante Laboratorio Multi...",Gran Buenos Aires,Full-time,Senior / Semi-Senior,Q - Analista de Aseguramiento de Calidad - Téc...,Farmacia industrial,Grupo Gestión,77
5186,<p>Nos encontramos en la búsqueda de un <stron...,Gran Buenos Aires,Full-time,Senior / Semi-Senior,Medico Coordinador con experiencia en UTI/ UCI,Emergentología,GMS S.A.,77
